# 정적 웹 크롤링과 스크래핑을 이용한 공공데이터와 SNS 데이터 수집

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
import io

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
savename = "C:/Temp/forecast.xml"
req.urlretrieve(url, savename)

xml = open(savename, "r", encoding="utf-8").read()
soup = BeautifulSoup(xml, 'xml') # xml

info = {}
for location in soup.find_all("location"):
    loc = location.find('city').string
    min_w = location.find_all('tmn')
    max_w = location.find_all('tmx')
    weather = [a.string+"~"+b.string for a, b in zip(min_w, max_w)]

    if not (loc in info):
        info[loc] = []
    for data in weather:
        info[loc].append(data)
print(info)

with open('C:/Temp/forecast.txt', "wt", encoding="utf-8") as f:
    for loc in sorted(info.keys()):
        f.write(str(loc)+'\n')
        for name in info[loc]:
            f.write('\t'+str(name)+'\n')

In [ ]:
import datetime
from bs4 import BeautifulSoup
import urllib.request

now = datetime.datetime.now()
nowDate = now.strftime('%Y년 %m월 %d일 %H시 %M분 입니다.')
print(nowDate)
# 오늘의 날씨
print('[ 오늘의 날씨 요약 ]')
webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('서울날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 서울 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('광주날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 광주 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

webpage = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('제주도날씨'))
soup = BeautifulSoup(webpage, 'html.parser')
temp = soup.select('div.temperature_text > strong')
cast = soup.select('span.weather.before_slash')
print(f'--> 제주도 날씨 : {temp[0].contents[1]}℃ {cast[0].text}')

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req

busNum = '360'
key = '%2BjzsSyNtwmcqxUsGnflvs3rW2oceFvhHR8AFkM3ao%2Fw50hwHXgGyPVutXw04uAXvrkoWgkoScvvhlH7jgD4%2FRQ%3D%3D'
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum
res = req.urlopen(url1)

soup = BeautifulSoup(res.read().decode('utf-8'), "xml")

busRouteId = None
for itemList in soup.find_all('itemList') :
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey='+key+'&busRouteId='+busRouteId
res = req.urlopen(url2)
soup = BeautifulSoup(res.read(), "xml")

busPos = []
for itemList in soup.find_all('itemList') :
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string

    busPos.append((gpsY, gpsX))

print('[ ' + busNum + '번 버스의 운행 위치 ]')
if len(busPos) != 0 :
    print(busNum + '번 버스 ' + str(len(busPos)) + '대 운행중...')
    for lat,lng in busPos :
        print(lat+','+lng)
else :
    print('현재 운행중인 ' + busNum + '번 버스가 없어요...')

print(url1)
print(url2)

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req

key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '100'
url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/LampScpgmtb/'+startIndex+'/'+endIndex+'/'
res = req.urlopen(url)
xml = res.read()
soup = BeautifulSoup(xml, "xml")

pjList = []
for itemList in soup.find_all('row') :
    up_nm = itemList.find('UP_NM').string
    up_nm = '없음' if up_nm is None else up_nm
    pgm_nm = itemList.find('PGM_NM').string
    pgm_nm = '없음' if pgm_nm is None else pgm_nm
    target_nm = itemList.find('TARGET_NM').string
    target_nm = '없음' if target_nm is None else target_nm
    u_price = itemList.find('U_PRICE').string
    u_price = '없음' if u_price is None else u_price
    pjList.append((up_nm, pgm_nm, target_nm, u_price))

print('[ 서울 청소년 수련관 강좌 리스트 ]')
for up_nm,pgm_nm,target_nm,u_price in pjList :
    print(up_nm+','+pgm_nm+','+target_nm+','+str(u_price))

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
import io

key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '100'
date = '20220815'

url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/CardSubwayStatsNew/'+startIndex+'/'+endIndex+'/'+date+'/'
print(url)
res = req.urlopen(url)
xml = res.read()

soup = BeautifulSoup(xml, "xml")

subwayList = []
for itemList in soup.find_all('row') :
    line_num = itemList.find('LINE_NUM').string
    sub_sta_nm = itemList.find('SUB_STA_NM').string
    ride_pasgr_num = itemList.find('RIDE_PASGR_NUM').string
    alight_pasgr_num = itemList.find('ALIGHT_PASGR_NUM').string
    subwayList.append((line_num, sub_sta_nm, ride_pasgr_num, alight_pasgr_num))

print('[ 서울시 지하철호선별 역별 승하차 인원 정보 ]')
for line_num, sub_sta_nm, ride_pasgr_num, alight_pasgr_num in subwayList :
    print(line_num+','+sub_sta_nm+','+ride_pasgr_num+','+alight_pasgr_num)


In [ ]:
import urllib.request
import json
client_key = 'izGsqP2exeThwwEUVU3x'
client_secret = 'WrwbQ1l6ZI'
query = '스마트폰'
encText = urllib.parse.quote_plus(query)
num = 100
naver_url = 'https://openapi.naver.com/v1/search/blog.json?query=' + encText + '&display=' + str(num)

request = urllib.request.Request(naver_url)
request.add_header("X-Naver-Client-Id",client_key)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    dataList = json.loads(response_body)

    print('[' + query + '에 대한 네이버 블로그 글 ]')
    for count, data in enumerate(dataList['items'], 1) :
        print (str(count) + ' : ' + data['title'])
        print ('[' + data['description'] + ']')
else:
    print('오류 코드 : ' + rescode)

In [ ]:
dataList # JSON

In [ ]:
dataList["items"][0]["description"]

In [ ]:
title = []
desc = []
for data in dataList["items"]:
    title.append(data["title"])
    desc.append(data["description"])  

In [ ]:
len(title)

In [ ]:
len(desc)

In [ ]:
title

In [ ]:
import re
for i in range(len(title)) :
   title[i] = re.sub("</?b>", "", title[i])

In [ ]:
for i in range(len(desc)) :
   desc[i] = re.sub("</?b>", "", desc[i])

In [ ]:
import pandas as pd
dictdata = {"title":title,"description":desc}
df = pd.DataFrame(dictdata)
df

In [ ]:
df.to_csv("output/smartphone.csv")

In [ ]:
import urllib.request
import json

client_key = 'izGsqP2exeThwwEUVU3x'
client_secret = 'WrwbQ1l6ZI'
query = '오징어게임'
encText = urllib.parse.quote_plus(query)

num = 100
naver_url = 'https://openapi.naver.com/v1/search/news.json?query=' + encText + '&display=' + str(num)

request = urllib.request.Request(naver_url)
request.add_header("X-Naver-Client-Id",client_key)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)

rescode = response.getcode()

if(rescode == 200):
    response_body = response.read()
    dataList = json.loads(response_body)
    count = 1
    print('[' + query + '에 대한 네이버 뉴스 글 ]')
    for data in dataList['items'] :
        print (str(count) + ' : ' + data['title'])
        print ('[' + data['description'] + ']')
        count += 1
else:
    print('오류 코드 : ' + rescode)
